In [1]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.utils import shuffle
pd.set_option("max_columns",None)

In [2]:
# load the dataset created by the songs-data-processyng.ipynb to avoid
full_tracks = pd.read_csv('full_tracks.csv')
#full_tracks.shape
full_tracks.head()

,track_id,album_date_created,yt_views,genre_Experimental,genre_Electronic,genre_Rock,genre_Instrumental,genre_Pop,genre_Folk,genre_Punk,genre_Avant-Garde,genre_Hip-Hop,genre_Noise,track_id.1,chroma_cens_kurtosis_1,chroma_cens_kurtosis_2,chroma_cens_kurtosis_3,chroma_cens_kurtosis_4,chroma_cens_kurtosis_5,chroma_cens_kurtosis_6,chroma_cens_kurtosis_7,chroma_cens_kurtosis_8,chroma_cens_kurtosis_9,chroma_cens_kurtosis_10,chroma_cens_kurtosis_11,chroma_cens_kurtosis_12,chroma_cens_max_1,chroma_cens_max_2,chroma_cens_max_3,chroma_cens_max_4,chroma_cens_max_5,chroma_cens_max_6,chroma_cens_max_7,chroma_cens_max_8,chroma_cens_max_9,chroma_cens_max_10,chroma_cens_max_11,chroma_cens_max_12,chroma_cens_mean_1,chroma_cens_mean_2,chroma_cens_mean_3,chroma_cens_mean_4,chroma_cens_mean_5,chroma_cens_mean_6,chroma_cens_mean_7,chroma_cens_mean_8,chroma_cens_mean_9,chroma_cens_mean_10,chroma_cens_mean_11,chroma_cens_mean_12,chroma_cens_median_1,chroma_cens_median_2,chroma_cens_median_3,chroma_cens_median_4,chroma_cens_median_5,chroma_cens_median_6,chroma_cens_median_7,chroma_cens_median_8,chroma_cens_median_9,chroma_cens_median_10,chroma_cens_median_11,chroma_cens_median_12,chroma_cens_min_1,chroma_cens_min_2,chroma_cens_min_3,chroma_cens_min_4,chroma_cens_min_5,chroma_cens_min_6,chroma_cens_min_7,chroma_cens_min_8,chroma_cens_min_9,chroma_cens_min_10,chroma_cens_min_11,chroma_cens_min_12,chroma_cens_skew_1,chroma_cens_skew_2,chroma_cens_skew_3,chroma_cens_skew_4,chroma_cens_skew_5,chroma_cens_skew_6,chroma_cens_skew_7,chroma_cens_skew_8,chroma_cens_skew_9,chroma_cens_skew_10,chroma_cens_skew_11,chroma_cens_skew_12,chroma_cens_std_1,chroma_cens_std_2,chroma_cens_std_3,chroma_cens_std_4,chroma_cens_std_5,chroma_cens_std_6,chroma_cens_std_7,chroma_cens_std_8,chroma_cens_std_9,chroma_cens_std_10,chroma_cens_std_11,chroma_cens_std_12,chroma_cqt_kurtosis_1,chroma_cqt_kurtosis_2,chroma_cqt_kurtosis_3,chroma_cqt_kurtosis_4,chroma_cqt_kurtosis_5,chroma_cqt_kurtosis_6,chroma_cqt_kurtosis_7,chroma_cqt_kurtosis_8,chroma_cqt_kurtosis_9,chroma_cqt_kurtosis_10,chroma_cqt_kurtosis_11,chroma_cqt_kurtosis_12,chroma_cqt_max_1,chroma_cqt_max_2,chroma_cqt_max_3,chroma_cqt_max_4,chroma_cqt_max_5,chroma_cqt_max_6,chroma_cqt_max_7,chroma_cqt_max_8,chroma_cqt_max_9,chroma_cqt_max_10,chroma_cqt_max_11,chroma_cqt_max_12,chroma_cqt_mean_1,chroma_cqt_mean_2,chroma_cqt_mean_3,chroma_cqt_mean_4,chroma_cqt_mean_5,chroma_cqt_mean_6,chroma_cqt_mean_7,chroma_cqt_mean_8,chroma_cqt_mean_9,chroma_cqt_mean_10,chroma_cqt_mean_11,chroma_cqt_mean_12,chroma_cqt_median_1,chroma_cqt_median_2,chroma_cqt_median_3,chroma_cqt_median_4,chroma_cqt_median_5,chroma_cqt_median_6,chroma_cqt_median_7,chroma_cqt_median_8,chroma_cqt_median_9,chroma_cqt_median_10,chroma_cqt_median_11,chroma_cqt_median_12,chroma_cqt_min_1,chroma_cqt_min_2,chroma_cqt_min_3,chroma_cqt_min_4,chroma_cqt_min_5,chroma_cqt_min_6,chroma_cqt_min_7,chroma_cqt_min_8,chroma_cqt_min_9,chroma_cqt_min_10,chroma_cqt_min_11,chroma_cqt_min_12,chroma_cqt_skew_1,chroma_cqt_skew_2,chroma_cqt_skew_3,chroma_cqt_skew_4,chroma_cqt_skew_5,chroma_cqt_skew_6,chroma_cqt_skew_7,chroma_cqt_skew_8,chroma_cqt_skew_9,chroma_cqt_skew_10,chroma_cqt_skew_11,chroma_cqt_skew_12,chroma_cqt_std_1,chroma_cqt_std_2,chroma_cqt_std_3,chroma_cqt_std_4,chroma_cqt_std_5,chroma_cqt_std_6,chroma_cqt_std_7,chroma_cqt_std_8,chroma_cqt_std_9,chroma_cqt_std_10,chroma_cqt_std_11,chroma_cqt_std_12,chroma_stft_kurtosis_1,chroma_stft_kurtosis_2,chroma_stft_kurtosis_3,chroma_stft_kurtosis_4,chroma_stft_kurtosis_5,chroma_stft_kurtosis_6,chroma_stft_kurtosis_7,chroma_stft_kurtosis_8,chroma_stft_kurtosis_9,chroma_stft_kurtosis_10,chroma_stft_kurtosis_11,chroma_stft_kurtosis_12,chroma_stft_max_1,chroma_stft_max_2,chroma_stft_max_3,chroma_stft_max_4,chroma_stft_max_5,chroma_stft_max_6,chroma_stft_max_7,chroma_stft_max_8,chroma_stft_max_9,chroma_stft_max_10,chroma_stft_max_11,chroma_stft_max_12,chroma_stft_mean_1,chroma_stft_mean_2,chroma_stft_mean_3,chroma_stft_mean_4,chroma_stft_m

In [3]:
full_tracks = full_tracks.drop('track_id.1', axis=1)
full_tracks = full_tracks.sort_values('album_date_created')

In [4]:
# check how many songs about the popularity threshold (view_th)
view_th = 5000000
len(full_tracks[full_tracks['yt_views']>view_th])

# we use songs with date > 2016 to test
test_split = full_tracks[full_tracks['album_date_created']>='2016']

display(len(test_split))
# these are the popular songs on the test datas
test_pop = test_split[test_split['yt_views']>view_th]
test_pop_len = len(test_pop)
display(test_pop_len)
# these are the no-popular songs on the test datas
test_nopop = test_split[test_split['yt_views']<=view_th].tail(test_pop_len)
display(len(test_nopop))

test_split2 = pd.concat([test_pop, test_nopop])
test_split2 = test_split2.drop(columns='album_date_created')
test_split2 = shuffle(test_split2) # mixing the rows
X_test = test_split2.drop(columns='yt_views')
Y_test = test_split2['yt_views']

4996

731

731

In [5]:
# creat validation data 50-50 pop and nopop, and same length as test data
valid_split = full_tracks[full_tracks['album_date_created']<'2016']
valid_pop = valid_split[valid_split['yt_views']>view_th].tail(test_pop_len)
valid_pop_last_date = valid_pop['album_date_created'].head(1)
display('valid_pop_last_date: '+valid_pop_last_date)
valid_nopop = valid_split[valid_split['yt_views']<=view_th].tail(test_pop_len)
valid_nopop_last_date = valid_nopop['album_date_created'].head(1)
display('valid_nopop_last_date: '+valid_nopop_last_date)
display(len(valid_pop))
display(len(valid_nopop))

valid_split2 = pd.concat([valid_pop, valid_nopop])
valid_split2 = valid_split2.drop(columns='album_date_created')
valid_split2 = shuffle(valid_split2) # mixing the rows
X_valid = valid_split2.drop(columns='yt_views')
Y_valid = valid_split2['yt_views']

31615    valid_pop_last_date: 2015-02-12 08:15:34
Name: album_date_created, dtype: object

34707    valid_nopop_last_date: 2015-10-22 12:45:23
Name: album_date_created, dtype: object

731

731

In [6]:
# make the train data from the remaining
train_split = full_tracks[full_tracks['album_date_created']<'2015-02-12 08:15:34']
train_pop = train_split[train_split['yt_views']>view_th]
train_pop_len = len(train_pop)
display(train_pop_len)
train_nopop = train_split[train_split['yt_views']<=view_th].tail(train_pop_len)
display(len(train_nopop))
display('pop_last_date: '+train_pop['album_date_created'].head(1))
display('nopop_last_date: '+train_nopop['album_date_created'].head(1))

train_split2 = pd.concat([train_pop, train_nopop])
train_split2 = train_split2.drop(columns='album_date_created')
train_split2 = shuffle(train_split2) # mixing the rows

X_train = train_split2.drop(columns='yt_views')

Y_train = train_split2['yt_views']

5230

5230

8    pop_last_date: 2008-11-26 01:44:45
Name: album_date_created, dtype: object

25330    nopop_last_date: 2013-08-05 10:10:09
Name: album_date_created, dtype: object

In [7]:
from sklearn.preprocessing import StandardScaler

# Use standard scaler to standardize features by removing the mean and scaling to unit variance.

scaler = StandardScaler().fit(X_train)

X_train = scaler.transform(X_train)
X_valid = scaler.transform(X_valid)
X_test  = scaler.transform(X_test)

# Needed to reshape the 1D arrays to 2D arrays.


Y_train = Y_train.values.reshape(-1, 1)
Y_valid = Y_valid.values.reshape(-1, 1)
Y_test = Y_test.values.reshape(-1, 1)


scaler = StandardScaler().fit(Y_train)

Y_train = scaler.transform(Y_train)
Y_valid = scaler.transform(Y_valid)
Y_test  = scaler.transform(Y_test)

display(X_train)

C:\Users\Lehel\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Users\Lehel\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:7: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  import sys
C:\Users\Lehel\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:8: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  
C:\Users\Lehel\Anaconda3\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':


array([[-1.27515798,  1.33294614, -0.66450403, ..., -0.53092035,
         0.64912925, -0.62552837],
       [-2.07565368, -0.75021786, -0.66450403, ...,  0.26719877,
        -0.35578621, -0.52363868],
       [ 0.36634312, -0.75021786, -0.66450403, ..., -0.53092035,
        -0.79206896, -0.87082379],
       ...,
       [-1.0372501 , -0.75021786, -0.66450403, ...,  1.74941998,
         0.48998032,  0.2995008 ],
       [-1.32470329, -0.75021786,  1.50488178, ..., -0.53092035,
         0.20474913, -0.7143501 ],
       [ 0.90287824,  1.33294614, -0.66450403, ...,  0.95130087,
        -0.68505268, -0.83055663]])

In [8]:
# Recursive feature elimination
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier


model_rfe = Lasso()

rfe = RFE(model_rfe, 30)
fit_rfe = rfe.fit(X_train, Y_train)
print("Num Features: ", fit_rfe.n_features_)
print("Selected Features: ", fit_rfe.support_)
print("Feature Ranking: ", fit_rfe.ranking_)

X_train_selected_columns = X_train[:, fit_rfe.support_]

display(X_train_selected_columns)

C:\Users\Lehel\Anaconda3\envs\tensorflow\lib\site-packages\sklearn\utils\validation.py:752: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


Num Features:  30
Selected Features:  [False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False False False False False False False False False False False
 False False 

array([[-0.03595105, -0.93807143, -0.68969603, ..., -0.53092035,
         0.64912925, -0.62552837],
       [-1.34429771,  2.59274541, -1.19909314, ...,  0.26719877,
        -0.35578621, -0.52363868],
       [-1.76028399,  0.42904561,  1.258237  , ..., -0.53092035,
        -0.79206896, -0.87082379],
       ...,
       [-0.73780524,  1.43020996,  1.74266372, ...,  1.74941998,
         0.48998032,  0.2995008 ],
       [ 1.14798032,  1.9463431 ,  1.70048974, ..., -0.53092035,
         0.20474913, -0.7143501 ],
       [ 1.40811708,  0.47055443, -0.59720342, ...,  0.95130087,
        -0.68505268, -0.83055663]])

In [9]:
X_valid_selected_columns = X_valid[:, fit_rfe.support_]
X_test_selected_columns = X_test[:, fit_rfe.support_]

In [10]:
display(X_train_selected_columns.shape)

(10460, 30)

In [15]:
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping

def song_model(x_train, y_train, x_val, y_val, params):
    model = Sequential()

    model.add(Dense(params['first_neuron'], input_dim=X_train_selected_columns.shape[1], activation=params['activation']))
    model.add(Dropout(params['dropout']))
    model.add(Dense(1, activation=params['last_activation']))

    # We also tried different loss functions and optimizers in order to achieve better performance.
    model.compile(loss=params['losses'], optimizer = params['optimizer'], metrics=['mse'])
    # Set callback functions to early stop training and save the best model so far
    callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=2)]
    # Fit the model
    out = model.fit(x_train, y_train, epochs = params['epochs'], validation_data=(x_val, y_val), batch_size=params['batch_size'],
                   callbacks = callbacks, verbose = 2)
    
    return out, model

In [16]:
# Hyperparameter optimization
import talos
from keras.activations import relu, elu, softmax
from keras.losses import mse, logcosh



p = {
    'first_neuron': [128, 500],
    'activation': [relu, elu],
    'last_activation': ['relu', 'sigmoid'],
    'optimizer': ['adam'],
    'batch_size': [16, 32],
    'losses': [mse, logcosh],
    'epochs': [50],
    'dropout': (0, 0.40, 10)
    }

t = talos.Scan(X_train_selected_columns, Y_train,
               x_val = X_valid_selected_columns,
               y_val = Y_valid,
               params = p,
               model = song_model,
               grid_downsample = .1)


  0%|                                                                                           | 0/32 [00:00<?, ?it/s]

Train on 10460 samples, validate on 1462 samples
Epoch 1/50
 - 2s - loss: 0.1594 - mean_squared_error: 1.0235 - val_loss: 0.1872 - val_mean_squared_error: 1.2116
Epoch 2/50
 - 1s - loss: 0.1422 - mean_squared_error: 0.9999 - val_loss: 0.1866 - val_mean_squared_error: 1.2125
Epoch 3/50
 - 1s - loss: 0.1418 - mean_squared_error: 0.9999 - val_loss: 0.1864 - val_mean_squared_error: 1.2126



  3%|██▌                                                                                | 1/32 [00:04<02:23,  4.63s/it]

Train on 10460 samples, validate on 1462 samples
Epoch 1/50
 - 1s - loss: 1.0035 - mean_squared_error: 1.0035 - val_loss: 1.2128 - val_mean_squared_error: 1.2128
Epoch 2/50
 - 1s - loss: 1.0028 - mean_squared_error: 1.0028 - val_loss: 1.2128 - val_mean_squared_error: 1.2128
Epoch 3/50
 - 1s - loss: 1.0000 - mean_squared_error: 1.0000 - val_loss: 1.2128 - val_mean_squared_error: 1.2128



  6%|█████▏                                                                             | 2/32 [00:08<02:15,  4.50s/it]

...

Train on 10460 samples, validate on 1462 samples
Epoch 1/50
 - 2s - loss: 0.1534 - mean_squared_error: 1.0331 - val_loss: 0.1863 - val_mean_squared_error: 1.2128
Epoch 2/50
 - 1s - loss: 0.1419 - mean_squared_error: 1.0005 - val_loss: 0.1863 - val_mean_squared_error: 1.2128
Epoch 3/50
 - 1s - loss: 0.1418 - mean_squared_error: 1.0003 - val_loss: 0.1863 - val_mean_squared_error: 1.2128



 97%|███████████████████████████████████████████████████████████████████████████████▍  | 31/32 [03:07<00:05,  5.76s/it]

Train on 10460 samples, validate on 1462 samples
Epoch 1/50
 - 3s - loss: 1.0005 - mean_squared_error: 1.0005 - val_loss: 1.2128 - val_mean_squared_error: 1.2128
Epoch 2/50
 - 2s - loss: 0.9999 - mean_squared_error: 0.9999 - val_loss: 1.2128 - val_mean_squared_error: 1.2128
Epoch 3/50
 - 2s - loss: 1.0006 - mean_squared_error: 1.0006 - val_loss: 1.2128 - val_mean_squared_error: 1.2128



100%|██████████████████████████████████████████████████████████████████████████████████| 32/32 [03:15<00:00,  6.31s/it]


Scan Finished!


In [26]:
# Reconstruct best model(problems with talos).

best_model = Sequential()

best_model.add(Dense(10, input_dim=X_train_selected_columns.shape[1], activation='relu'))
best_model.add(Dropout(0.5))
best_model.add(Dense(100, activation='relu'))
best_model.add(Dense(1, activation='sigmoid'))

best_model.compile(loss='mse', optimizer = 'adam', metrics=['mse'])
callbacks = [EarlyStopping(monitor='val_mean_squared_error', patience=2)]

best_model.fit(X_train_selected_columns, Y_train, epochs = 50,
          validation_data=(X_valid_selected_columns, Y_valid), batch_size=16,
          callbacks = callbacks, verbose = 2)
    

Train on 10460 samples, validate on 1462 samples
Epoch 1/50
 - 4s - loss: 1.0193 - mean_squared_error: 1.0193 - val_loss: 1.2110 - val_mean_squared_error: 1.2110
Epoch 2/50
 - 3s - loss: 1.0009 - mean_squared_error: 1.0009 - val_loss: 1.2120 - val_mean_squared_error: 1.2120
Epoch 3/50
 - 3s - loss: 1.0003 - mean_squared_error: 1.0003 - val_loss: 1.2124 - val_mean_squared_error: 1.2124


In [28]:
e = best_model.evaluate(X_train_selected_columns, Y_train)
e

10460/10460 [==============================] - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - ETA:  - 0s 37us/step


[0.9997901514087079, 0.9997901514087079]